# Extract all tables in `sales` schema

In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
from delta.tables import DeltaTable
import json
from datetime import datetime, timedelta


In [3]:
with open('config.json', 'r') as f:
    config = json.load(f)

## Control variables

In [21]:
schema = config["general"]["schema"]
catalog = f'{config["general"]["user"]}_raw'
db_password = dbutils.secrets.get(scope="antonio_junior_adw", key="pswd_mssql")
db_host = dbutils.secrets.get(scope="antonio_junior_adw", key="ip_mssql")
db_port = dbutils.secrets.get(scope="antonio_junior_adw", key="port_mssql")
db_user = config["general"]["db_user"]
database =  config["general"]["database"]
tables_list = list(config["tables"].keys())
last_modified_date = datetime.today() - timedelta(days=1)
last_modified_date = last_modified_date.strftime('%Y-%m-%d 00:00:00')


Connections vars

In [19]:
jdbc_url = f"jdbc:sqlserver://{db_host}:{db_port};databaseName={database};encrypt=true;trustServerCertificate=true;"

connection_properties = {
    "user": db_user, 
    "password": db_password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# SQL QUERIES

List tables in schema

Create schema

In [20]:
create_schema = f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}"

## Extraction steps

Extract all data from each table.

In [ ]:
for table_name in tables_list:
    delta_table_name = f"{catalog}.{schema}.{table_name}"
    print(table_name)
    # SQL query to fetch incremental data
    query = f"(SELECT * FROM {schema}.{table_name} WHERE ModifiedDate > '{last_modified_date}') AS subquery"
    print(query)

    incremental_df = spark.read.jdbc(
        url=jdbc_url,
        table=query,
        properties=connection_properties
    )

    incremental_df.write.mode("overwrite").format("delta").saveAsTable(delta_table_name)

    # Check if the Delta table exists
    # try:
    #     delta_table = DeltaTable.forName(spark, delta_table_name)
    #     table_exists = True
    # except:
    #     table_exists = False
    
    # print(config["tables"][table_name])
    
    # if table_exists:
    #     # Perform the merge operation if the table exists
    #     delta_table.alias("target").merge(
    #         incremental_df.alias("source"),
    #         "target. = source.CustomerID"  # Use the appropriate unique identifier
    #     ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
    # else:
    #     # Create the Delta table if it doesn't exist
    #     incremental_df.write.format("delta").saveAsTable(delta_table_name)

    break

CountryRegionCurrency
(SELECT * FROM sales.CountryRegionCurrency WHERE ModifiedDate > '2025-01-19 00:00:00') AS subquery
